In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import linearmodels.iv.model as lm
from scipy import stats
from itertools import combinations

In [2]:
def firstStageRegress(IVs, endo):
    X_stage1 = sm.add_constant(df[IVs])
    y_stage1 = df[endo]  # Endogenous variable

    # Fit the first-stage regression to find the predicted values of p
    results_stage1 = sm.OLS(y_stage1, X_stage1).fit()

    # Predict the values of educ using the first-stage regression model
    predictor = results_stage1.predict(X_stage1)

    return results_stage1, predictor  # Return p_hat

def secondStageRegress(predictor, endo, exo):
    df["phat"] = predictor
    
    # Stage 2: Use the predicted values of p
    X_stage2 = sm.add_constant(df[["phat"] + exo])
    y_stage2 = df[endo]  

    # Fit the second-stage regression model
    results_stage2 = sm.OLS(y_stage2, X_stage2).fit()
    
    return results_stage2

In [3]:
df = pd.read_csv("Data-GP1-1(updated).csv")
df = df.drop('Fri', axis=1)
df = df.drop('Sat', axis=1)
df = df.drop('Sun', axis=1)
df

,Mon,Tue,Wed,Thu,Date,Month,Year,Stormy,Mixed,p,q,Rainy,Cold,Wind
0,1,0,0,0,2,12,91,1,0,-0.430783,8.994421,1,0,2.995732
1,0,1,0,0,3,12,91,1,0,0.000000,7.707063,0,0,2.995732
2,0,0,1,0,4,12,91,0,1,0.072321,8.350194,1,1,2.813411
3,0,0,0,1,5,12,91,1,0,0.247139,8.656955,0,1,3.036554
4,0,0,0,0,6,12,91,1,0,0.664327,7.844241,0,1,3.036554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,1,0,0,0,4,5,92,0,0,-0.798508,8.610683,0,0,2.862201
107,0,1,0,0,5,5,92,0,1,-0.087011,7.162397,0,0,2.908721
108,0,0,1,0,6,5,92,0,1,0.184922,7.362010,0,0,2.862201
109,0,0,0,1,7,5,92,0,1,0.223143,8.764053,0,0,2.813411


In [4]:
# Generate all possible combinations of the variables

combinations_list = []
variables = [col for col in df.columns if col not in ["q", "p"]]
    
for r in range(1, len(variables) + 1):
    combinations_list.extend(combinations(variables, r))

# Print the combinations as lists
for combination in combinations_list:
    combination_as_list = list(combination)
    
print(len(combinations_list))

4095


In [5]:
# Finding Potential IVs
sign_IVs = []
sign_phat = []

for combination in combinations_list:
    result, p_hat = firstStageRegress(list(combination), "p")
    
    if all(result.pvalues < 0.05):  # Check if all p-values are less than 0.05
        sign_IVs.append(list(combination))  # Append the combination as a list
        sign_phat.append(p_hat)
        
for combination in sign_IVs:
    print(combination)
print(f"Total: {len(sign_IVs)}")

['Year']
['Stormy']
['Cold']
['Wind']
['Date', 'Cold']
['Month', 'Year']
['Year', 'Stormy']
['Year', 'Cold']
['Stormy', 'Mixed']
['Date', 'Month', 'Year']
['Date', 'Year', 'Cold']
Total: 11


In [6]:
# Finding suitable Exogeneous variables with respect to identified IVs
feasible_combis = []

for combination in combinations_list:
    for idx, predictor in enumerate(sign_phat):
        result = secondStageRegress(predictor, "q",list(combination))

        if all(result.pvalues < 0.05):  # Check if all p-values are less than 0.05           
            if all(item not in combination for item in sign_IVs[idx]):
                feasible_combis.append({
                    "IVs": sign_IVs[idx],
                    "Exo": list(combination)
                })
    
for combination in feasible_combis:
    print(combination)

print(f"Total:{len(feasible_combis)}")

{'IVs': ['Wind'], 'Exo': ['Mon']}
{'IVs': ['Stormy'], 'Exo': ['Tue']}
{'IVs': ['Wind'], 'Exo': ['Tue']}
{'IVs': ['Stormy', 'Mixed'], 'Exo': ['Tue']}
{'IVs': ['Stormy'], 'Exo': ['Wed']}
{'IVs': ['Wind'], 'Exo': ['Wed']}
{'IVs': ['Stormy', 'Mixed'], 'Exo': ['Wed']}
{'IVs': ['Date', 'Month', 'Year'], 'Exo': ['Wind']}
{'IVs': ['Stormy'], 'Exo': ['Mon', 'Thu']}
{'IVs': ['Wind'], 'Exo': ['Mon', 'Thu']}
{'IVs': ['Stormy', 'Mixed'], 'Exo': ['Mon', 'Thu']}
{'IVs': ['Date', 'Month', 'Year'], 'Exo': ['Mon', 'Wind']}
{'IVs': ['Stormy'], 'Exo': ['Tue', 'Wed']}
{'IVs': ['Wind'], 'Exo': ['Tue', 'Wed']}
{'IVs': ['Stormy', 'Mixed'], 'Exo': ['Tue', 'Wed']}
{'IVs': ['Date', 'Month', 'Year'], 'Exo': ['Tue', 'Wind']}
{'IVs': ['Date', 'Month', 'Year'], 'Exo': ['Wed', 'Wind']}
{'IVs': ['Date', 'Month', 'Year'], 'Exo': ['Mon', 'Thu', 'Wind']}
{'IVs': ['Date', 'Month', 'Year'], 'Exo': ['Tue', 'Wed', 'Stormy']}
{'IVs': ['Date', 'Month', 'Year'], 'Exo': ['Tue', 'Wed', 'Wind']}
Total:20


In [7]:
best_combi = []

for combination in feasible_combis:    
    mlr2 = lm.IV2SLS(dependent=df["q"], 
                     exog=df[combination["Exo"]], 
                     endog=df["p"], 
                     instruments=df[combination["IVs"]]).fit(cov_type="homoskedastic", debiased=True)

    if mlr2.wu_hausman().pval < 0.05:
        combination["Sargan"] = mlr2.sargan.pval
        best_combi.append(combination)
        
# Sort best_combi in ascending order based on "Sargan" values
best_combi = sorted(best_combi, key=lambda x: x["Sargan"])

for combination in best_combi:
    print(combination)

print(f"Total:{len(best_combi)}")

{'IVs': ['Wind'], 'Exo': ['Mon'], 'Sargan': nan}
{'IVs': ['Stormy'], 'Exo': ['Tue'], 'Sargan': nan}
{'IVs': ['Wind'], 'Exo': ['Tue'], 'Sargan': nan}
{'IVs': ['Stormy', 'Mixed'], 'Exo': ['Tue'], 'Sargan': 4.440892098500626e-16}
{'IVs': ['Stormy'], 'Exo': ['Wed'], 'Sargan': nan}
{'IVs': ['Wind'], 'Exo': ['Wed'], 'Sargan': nan}
{'IVs': ['Stormy', 'Mixed'], 'Exo': ['Wed'], 'Sargan': 1.0769163338864018e-14}
{'IVs': ['Date', 'Month', 'Year'], 'Exo': ['Wind'], 'Sargan': 0.005366629215837526}
{'IVs': ['Stormy'], 'Exo': ['Mon', 'Thu'], 'Sargan': nan}
{'IVs': ['Wind'], 'Exo': ['Mon', 'Thu'], 'Sargan': nan}
{'IVs': ['Stormy', 'Mixed'], 'Exo': ['Mon', 'Thu'], 'Sargan': 3.876432508320704e-11}
{'IVs': ['Date', 'Month', 'Year'], 'Exo': ['Mon', 'Wind'], 'Sargan': 0.005024560395952005}
{'IVs': ['Stormy'], 'Exo': ['Tue', 'Wed'], 'Sargan': nan}
{'IVs': ['Wind'], 'Exo': ['Tue', 'Wed'], 'Sargan': nan}
{'IVs': ['Stormy', 'Mixed'], 'Exo': ['Tue', 'Wed'], 'Sargan': 0.003034736632275825}
{'IVs': ['Date', 'Mont

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.227
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     15.83
Date:                Mon, 18 Sep 2023   Prob (F-statistic):           9.32e-07
Time:                        19:47:28   Log-Likelihood:                -35.891
No. Observations:                 111   AIC:                             77.78
Df Residuals:                     108   BIC:                             85.91
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3919      0.051     -7.756      0.0

In [9]:
 mlr2 = lm.IV2SLS(dependent=df["q"], 
                 exog=df[["Mon","Tue","Wed","Thu"]], 
                 endog=df["p"], 
                 instruments=df[["Stormy", "Mixed"]]).fit(cov_type="homoskedastic", debiased=True)
    
print(mlr2.wu_hausman(),"\n")
print(mlr2.sargan)

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 29.1437
P-value: 0.0000
Distributed: F(1,105) 

Sargan's test of overidentification
H0: The model is not overidentified.
Statistic: 3.1727
P-value: 0.0749
Distributed: chi2(1)
